In [14]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [12]:
openai.api_key  = os.environ['OPENAI_API_KEY']

KeyError: 'OPENAI_API_KEY'

In [2]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/ai_adoption_framework_whitepaper.pdf")

In [3]:
pages = loader.load()

In [4]:
len(pages)

37

In [5]:
page = pages[0]
page

Document(metadata={'source': 'data/ai_adoption_framework_whitepaper.pdf', 'page': 0}, page_content='Google Cloud’s \nAI Adoption \nFramework \n')

In [6]:
page.metadata

{'source': 'data/ai_adoption_framework_whitepaper.pdf', 'page': 0}

In [7]:
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader

In [9]:
url="https://www.youtube.com/watch?v=rXIYVvoSoDY"
save_dir="docs/youtube/"
loader = GenericLoader(
    YoutubeAudioLoader([url],save_dir),
    OpenAIWhisperParser()
)
docs = loader.load()

[youtube] Extracting URL: https://www.youtube.com/watch?v=rXIYVvoSoDY
[youtube] rXIYVvoSoDY: Downloading webpage
[youtube] rXIYVvoSoDY: Downloading ios player API JSON
[youtube] rXIYVvoSoDY: Downloading mweb player API JSON
[youtube] rXIYVvoSoDY: Downloading m3u8 information
[info] rXIYVvoSoDY: Downloading 1 format(s): 140
[download] docs/youtube//OSHO： Only These People Have Lived....m4a has already been downloaded
[download] 100% of    1.96MiB
[ExtractAudio] Not converting audio docs/youtube//OSHO： Only These People Have Lived....m4a; file is already in target format m4a
Transcribing part 1!
Attempt 1 failed. Exception: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-ke

In [15]:
docs[0].page_content[0:500]

NameError: name 'docs' is not defined

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

In [16]:
chunk_size =26
chunk_overlap = 4

In [17]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

In [18]:
text1 = 'This is a test sentence. This is another test sentence. This is a third test sentence.'

In [19]:
r_splitter.split_text(text1)

['This is a test sentence.',
 'This is another test',
 'sentence. This is a third',
 'test sentence.']

In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader
import os
import sys

In [2]:

# Load PDF
loaders = [
    # Duplicate documents on purpose - messy data
    PyPDFLoader("data/ai_adoption_framework_whitepaper.pdf"),
    PyPDFLoader("data/profile.pdf"),
    PyPDFLoader("data/prog.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [3]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [4]:
splits = text_splitter.split_documents(docs)

In [5]:
len(splits)

116

In [6]:
embedding = HuggingFaceEmbeddings()

/Users/rafikkouissar/opt/anaconda3/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-10-21 20:15:20.964340: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
sentence1 = "i like dogs"
sentence2 = "i like canines"
sentence3 = "the weather is ugly outside, but walking my dog made tolerable"

In [14]:
embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

In [9]:
import numpy as np

In [10]:
np.dot(embedding1, embedding2)

0.8981182464688037

In [11]:
np.dot(embedding1, embedding3)

0.03641027702126909

In [15]:
np.dot(embedding2, embedding3)

0.20956271656863232

In [16]:
from langchain.vectorstores import Chroma

In [17]:
persist_directory = 'docs/chroma/'

In [18]:
!rm -rf ./docs/chroma  # remove old database files if any

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [54]:
print(vectordb._collection.count())

116


In [21]:
question = "is there an email i can ask for help"

In [28]:
question = "what is flood insurance?"

In [33]:
docs = vectordb.similarity_search(question,k=2)

In [34]:
len(docs)

2

In [35]:
docs[0].page_content

'caused by flood. Ask your producer (who is your agent or broker) or\nyour insurance company about flood coverage. Visit the National\nFloodInsuranceProgram(ªNFIPº)at www.floodsmart.gov orby\nphone888-379-9531tolearnmore.\nCoverage for Loss\nfrom Water that\nBacks up Through\nSewers and DrainsInsurance companies are required to offer you this coverage for your\nfull policy limits. You have a choice to purchase this coverage. You\nmay choose lower limits if available from your insurance company.\nContact your producer or your insurance company for additional\ninformation and costs.\nDeductibles Allpoliciesincludeadeductible.Thedeductibleistheamount you\nareresponsibleforonacoveredloss. Thedeductiblecanbea\nfixed dollar amount or it can be a percentage (%) of your dwelling\ncoverage limits. Some policies may include separate deductibles for\nperils such as hurricanes, named storms, wind or hail. Contact your\nproducer or your insurance company if you have questions about\ndeductibles.\nS

In [25]:
vectordb.persist()

/var/folders/gk/sccf81q96ml582bt9kq0v8rm0000gn/T/ipykernel_6358/3711397106.py:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [36]:
for doc in docs:
    print(doc.metadata)

{'page': 16, 'source': 'data/prog.pdf'}
{'page': 20, 'source': 'data/prog.pdf'}


In [37]:
print(vectordb._collection.count())

116


In [38]:
texts = [
    """The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).""",
    """A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.""",
    """A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.""",
]

In [39]:
smalldb = Chroma.from_texts(texts, embedding=embedding)

In [40]:
question = "Tell me about all-white mushrooms with large fruiting bodies"

In [41]:
smalldb.similarity_search(question, k=2)

[Document(metadata={}, page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(metadata={}, page_content='A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.')]

In [43]:
smalldb.max_marginal_relevance_search(question, k=2, fetch_k=3)

[Document(metadata={}, page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(metadata={}, page_content='A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.')]

In [50]:
from langchain_ollama.llms import OllamaLLM


In [51]:
model = OllamaLLM(model="orca-mini")


In [52]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

In [53]:
from langchain.chains import RetrievalQA

In [55]:
docs = vectordb.similarity_search(question,k=2)

In [56]:
len(docs)

2

In [57]:
print(docs[0].page_content)

Floors±Type&Percentage Carpet34%
SheetVinyl33%
Hardwood33%
Basementinhome 0
CeilingHeight 8ft.orless
Number of Rooms with Cathedral or Vaulted
Ceilings0
NumberofRoomswithCrown Moulding 0
PrimaryHeatingSource GASHOTAIR
NumberofFireplaces 0
FirePlaceType(s)
KitchenCountertopMaterial PlasticLaminate(Formica)
NumberofFullBaths
NumberofHalfBaths
ElectricalWiringType YES
CentralAirConditioning Central


In [58]:
question = "what is flood insurance"

In [60]:
qa_chain = RetrievalQA.from_chain_type(
    model,
    retriever=vectordb.as_retriever()
)

In [61]:
result = qa_chain({"query": question})

/var/folders/gk/sccf81q96ml582bt9kq0v8rm0000gn/T/ipykernel_6358/4094420968.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


In [62]:
result["result"]

' Visit the National Flood Insurance Program(ªNFIPº) at www.floodsmart.gov or by phone888-379-9531to learn more.'

In [65]:
question = "what is the telephone number of the insrance company?"

In [64]:
result = qa_chain({"query": question})
result["result"]

' Progressive Home, by Homesite is not associated with Homesite. The policyholder must check if the insurance was issued by a company within the Homesite Group Incorporated or a licensed provider of homeowners, renters and condominium insurance in their state. The Homesite Group Incorporated provides coverage for, and pays the claims associated with, these insurance policies.'

In [67]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [68]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    model,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [70]:
result = qa_chain({"query": question})
result["result"]

" I'm sorry, but there is no information in your question about an insurance company. Can you please provide more context or additional information to help me better understand your question so that I can provide a helpful answer?"

In [71]:
result["source_documents"][0]

Document(metadata={'page': 30, 'source': 'data/prog.pdf'}, page_content='HMI0021119 NoticeofInsuranceInformationPracticesProgressive Home, by Homesite\nUnderwritten by HOMESITE INSURANCE COMPANY OF THE MIDWEST\nP.O. Box 5300\nBinghamton,NY13902-9953\nTel.(866)960-8609Fax(877)273-2984\nNotice of Insurance\nInformation Practices\nIssued by HOMESITE\nINSURANCE COMPANY OF THE\nMIDWEST\nHomesite Group Incorporated uses information from many sources.  This assists us to fairly determine\neligibility for our programs and ensure accurate rates for all policies.  Using this information also speeds the\napplication process.\nHow we may collect, use and disclose this information is regulated by law, and we would like you to be\naware of our practices and how they may affect your privacy.\nFollowing is a description of the kinds of information we may collect, how we may collect it, and what is done\nwith the information once it has been collected.  We also describe how you can find out what inform

In [72]:
qa_chain_mr = RetrievalQA.from_chain_type(
    model,
    retriever=vectordb.as_retriever(),
    chain_type="refine"
)
result = qa_chain_mr({"query": question})
result["result"]

' The telephone number of the insurance company is (866)960-8609.'